In [76]:
# -*- coding: utf-8 -*-

import openpyxl
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
from numpy.linalg import inv
from scipy.stats import norm


def QQ(vec):
    v = np.sort(vec)
    q = stats.norm.ppf((np.arange(len(v))+1-0.375)/(len(v)+0.25))
    return v, q

def RB_fit(RB):
    RB_ord = np.sort(RB)/100
    th_q = stats.norm.ppf((np.arange(len(RB))+1-0.375)/(len(RB)+0.25))
    q = th_q
    l = np.log(RB_ord)
    h = np.log(1-RB_ord)
    d = (np.mean(l*l) - pow(np.mean(l),2))*(np.mean(h*h) - pow(np.mean(h),2)) - pow(np.mean(l*h) - np.mean(l)*np.mean(h),2)
    d1 = (np.mean(h*h) - pow(np.mean(h),2))*(np.mean(l*q) - np.mean(l)*np.mean(q))-(np.mean(l*h) - np.mean(l)*np.mean(h))*(np.mean(h*q) - np.mean(h)*np.mean(q))
    d2 = (np.mean(l*h) - np.mean(l)*np.mean(h))*(np.mean(l*q) - np.mean(l)*np.mean(q)) - (np.mean(l*l) - pow(np.mean(l),2))*(np.mean(h*q) - np.mean(h)*np.mean(q))
    if d1 < 0:
        a = 0
        b = -(np.mean(h*q) - np.mean(h)*np.mean(q))/(np.mean(h*h) - pow(np.mean(h),2))
    elif d2 < 0:
        a = (np.mean(l*q) - np.mean(l)*np.mean(q))/(np.mean(l*l) - pow(np.mean(l),2))
        b = 0
    else:
        a = d1/d
        b = d2/d
    c = np.mean(q) - a*np.mean(l) + b*np.mean(h)
    RB_t = a*l - b*h + c
    _, pv = stats.kstest(RB_t,'norm')
    return a, b, c, pv, RB_t, th_q

def TP_fit(TP):
    TP_ord = np.sort(TP)
    th_q = stats.norm.ppf((np.arange(len(TP))+1-0.375)/(len(TP)+0.25))
    q = th_q
    g = np.log(TP_ord)
    t = TP_ord
    d = (np.mean(g*g) - pow(np.mean(g),2))*(np.mean(t*t) - pow(np.mean(t),2)) - pow(np.mean(g*t) - np.mean(g)*np.mean(t),2)
    d1 = (np.mean(t*t) - pow(np.mean(t),2))*(np.mean(g*q) - np.mean(g)*np.mean(q))-(np.mean(g*t) - np.mean(g)*np.mean(t))*(np.mean(t*q) - np.mean(t)*np.mean(q))
    d2 = (np.mean(g*g) - pow(np.mean(g),2))*(np.mean(t*q) - np.mean(t)*np.mean(q))-(np.mean(t*g) - np.mean(t)*np.mean(g))*(np.mean(g*q) - np.mean(g)*np.mean(q))
    if d1 < 0:
        a = 0
        b = (np.mean(g*q) - np.mean(g)*np.mean(q))/(np.mean(g*g) - pow(np.mean(g),2))
    elif d2 < 0:
        a = (np.mean(t*q) - np.mean(t)*np.mean(q))/(np.mean(t*t) - pow(np.mean(t),2))
        b = 0
    else:
        a = d1/d
        b = d2/d
    c = np.mean(q) - a*np.mean(g) - b*np.mean(t)
    TP_t = a*g + b*t + c
    _, pv = stats.kstest(TP_t,'norm')
    return a, b, c, pv, TP_t, th_q

RD = openpyxl.load_workbook('data_prep_ver2.xlsx', data_only=True)
st = RD['Sheet1']

Data = np.zeros((1,5))
i = 2
value = st['B%d' %i].value
while value != None:
    Data = np.vstack((Data, np.array([st['B%d' %i].value, st['C%d' % i].value, st['D%d' %i].value, st['E%d' %i].value, st['FB%d' %i].value])))
    # PCI, RB, RNTI, TP, Timestamp
    i += 1
    value = st['B%d' %i].value
Data = Data[1:,:].astype(np.float64)
Data

array([[3.0000e+00, 4.0000e-01, 3.0000e+00, 5.3330e+01,        nan],
       [3.0000e+00, 6.9000e+00, 3.0000e+00, 6.2910e+02,        nan],
       [3.0000e+00, 1.0000e-01, 3.0000e+00, 2.0530e+01,        nan],
       ...,
       [3.9400e+02, 3.1600e+01, 9.0000e+00, 1.5645e+03,        nan],
       [3.9400e+02, 5.1300e+01, 1.0000e+01, 7.9981e+03,        nan],
       [3.9400e+02, 6.0400e+01, 1.0000e+01, 2.8960e+03,        nan]])

In [77]:
#length가 30 미만인 RNTI 삭제
len_per_rnti = np.array([len(Data[Data[:,2] == np.float(i), 1]) for i in range(20)])
selected_rnti = np.where(len_per_rnti>30)
Data = Data[np.where(np.isin(Data[:,2], selected_rnti)),]
Data = np.reshape(Data, (len(Data[0]),5))

3184

In [153]:
total_sample_size =1000 #10000
rnti_sample = np.random.uniform(np.min(selected_rnti), np.max(selected_rnti), total_sample_size)
RB_index = 1
TP_index = 3
total_info = np.zeros((total_sample_size, 13))

def SamplingperRnti(Data, fixed_RNTI):
    sample_sm = Data[Data[:,2] == np.floor(fixed_RNTI), 1:4]   # RNTI = N 인 RB
    sample_bg = Data[Data[:,2] == np.floor(fixed_RNTI)+1, 1:4]   # RNTI = N+1 인 RB
    #print(len(sample_sm))
    sample_size = np.minimum(50, np.minimum(len(sample_sm), len(sample_bg)))
    resid_prob = fixed_RNTI-np.floor(fixed_RNTI)
    sample_size_array = np.random.multinomial(sample_size, [1-resid_prob, resid_prob], size = 1)
    sample_rnd_sm = sample_sm[np.random.choice(len(sample_sm),sample_size_array[0,0]),]   #fixed RNTI에 해당하는 RB 중 sample_size_array[0]만큼 ramdom 추출
    sample_rnd_bg = sample_bg[np.random.choice(len(sample_bg),sample_size_array[0,1]),]   #fixed RNTI+1에 해당하는 RB 중 sample_size_array[1]만큼 ramdom 추출
    #print(sample_rnd_sm)
    sample_rnd_total = np.vstack((sample_rnd_sm, sample_rnd_bg))
    
    RB_sample = sample_rnd_total[:,0]
    TP_sample = np.round(sample_rnd_total[:,2], decimals=3)
    
    return(RB_sample, TP_sample)

In [174]:
### multivariate normal test using Henze-Zirkler method

def Tvariable(n, k, Data):
    S = np.cov(Data.T, bias = True)
    beta = (1/np.sqrt(2))*((n*(2*k+1)/4)**(1/(k+4)))
    C = n*(1 + 2*beta**2)**(-k/2)  
    B = 0; T = 0
    for i in range(n):
        A = 0
        for j in range(n):
            #print(np.matmul(np.matmul((Data[i]- Data[j]).transpose(), inv(S)), (Data[i]-Data[j])))
            A += np.exp(-(beta**2/2)*(np.matmul(np.matmul((Data[i]- Data[j]).transpose(), inv(S)), (Data[i]-Data[j]))))
        B = np.exp(-(beta**2/(2*(1+beta**2)))*(np.matmul(np.matmul((Data[i]- np.mean(Data, axis = 0)).transpose(), inv(S)), (Data[i]-np.mean(Data, axis = 0)))))
        T += (1/n)*A -2*(1 + beta**2)**(-k/2) * B
    #print(beta, A, B, C, T)
    return beta, T + C
        

def Moments(beta, k):
    w = (1+beta**2)*(1+3*beta**2)
    Expec_T = 1-((1 + 2*beta**2)**(-k/2)*(1 + k*beta**2/(1+2*beta**2) + k*(k+2)*beta**4/2/(1+2*beta**2)**2))
    Var_T = 2*(1 + 4*beta**2)**(-k/2) + 2*((1+2*beta**2)**(-k))*(1+ 2*k*beta**4/(1+2*beta**2)**2 + (3*k*(k+2)*beta**8)/(4*(1+2*beta**2)**4))
    -(4*w**(-k/2))*(1 + (3*k*beta**4/2/w) + (k*(k+2)*beta**8/2/w**2))
    #print(w, Expec_T, Var_T)
    return Expec_T, Var_T

def jointpval(n, k, Data):
    beta, T = Tvariable(n, k, Data)
    Expec_T, Var_T = Moments(beta, k)
    VZ = np.log(1+Var_T/(Expec_T**2))
    EZ = np.log(Expec_T) - VZ/2
    #print(VZ, EZ)
    stand_z = (np.log(T) - EZ)/(np.sqrt(VZ) + 10**(-9))
    joint_pval = 2*norm.cdf(-np.absolute(stand_z))
    return joint_pval


Y = np.random.normal(0,1, (50,2))
X = np.array([[1,2],[3,4]])
pval_test = jointpval(50, 2, np.matmul(X,Y.T).T)

print(pval_test)


0.8178295421121836


In [ ]:
def FindParams(fixed_rnti, Data):
    RB_rnd_sample, TP_rnd_sample = SamplingperRnti(Data, fixed_rnti)   # random한 RNTI에 대해 RB, TP random sampling
    a_RB, b_RB, c_RB, pv, rt, tq = RB_fit(RB_rnd_sample)
    if (pv > 0.05):
        RB_trans_data = rt
        a_TP, b_TP, c_TP, pv, rt, tq = TP_fit(TP_rnd_sample)
        if(pv > 0.05):
            TP_trans_data = rt
            total_trans_data = np.hstack((RB_trans_data.reshape(-1,1), TP_trans_data.reshape(-1,1)))   # transformed data로 total data 생성
            joint_pv = jointpval(len(TP_trans_data), 2, total_trans_data)
            print(joint_pv)
            if joint_pv > 0.05:   # joint normal 만족하는지
                print(joint_pv)
                total_params = np.array([fixed_rnti, a_RB, b_RB, c_RB, a_TP, b_TP, c_TP])
                print(total_params)
                return total_params
            else:
                return [0]
        else:
                return [0]
    else:
                return [0]
                
ParamArray = np.zeros((7))
for i in range(total_sample_size):
    fixed_rnti = rnti_sample[i]
    print(fixed_rnti)
    params = FindParams(fixed_rnti, Data)
    if len(params) == 7:
        ParamArray = np.vstack((ParamArray, params))
    else:
        continue
    #PCI 별로 나눌 필요: x
        

In [179]:
# TP_test_sm = Data[Data[:,2] == fixed_RNTI, TP_index]   # RNTI = N 인 TP
# TP_test_bg = Data[Data[:,2] == fixed_RNTI+1, TP_index]   # RNTI = N+1 인 TP

# sample_size = min(50, min(len(TP_test_sm), len(TP_test_bg)))
# resid_prob = fixed_RNTI-np.floor(fixed_RNTI)
# sample_size_array = np.random.multinomial(sample_size, [resid_prob, 1-resid_prob])


# TP_rnd_sm = TP_test_sm[np.random.choice(len(TP_test_sm),sample_size_array[0])]   #fixed RNTI에 해당하는 TP 중 sample_size_array[0]만큼 ramdom 추출
# TP_rnd_bg = TP_test_bg[np.random.choice(len(TP_test_bg),sample_size_arrat[1])]   #fixed RNTI+1에 해당하는 TP 중 sample_size_array[1]만큼 ramdom 추출
# TP_rnd_total = np.hstack(TP_rnd_sm, TP_rnd_bg)
# a, b, c, pv, rt, tq = TP_fit(TP_rnd_total)

# print('a=', a, 'b=', b, 'c=', c, 'pv=', pv)
# plt.figure(figsize=(10,10))
# f = plt.subplot(1, 1, 1)
# f.plot(tq, tt, color = 'tab:blue', linewidth = 0, marker='o', markersize = 6)
# print(f)

